In [8]:
import os
import pandas as pd
from dotenv import load_dotenv
from binance.client import Client

# Carrega o arquivo .env
load_dotenv()

# Acessa as variáveis
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

client = Client(api_key,api_secret)

In [9]:
# Todos os preços
todos_precos = client.get_all_tickers()

numero_tickers = len(todos_precos)
# print(todos_precos)

# Obter informações detalhadas para todos os tickers
all_ticker_info = client.get_ticker()
# for ticker in all_ticker_info:
#     print(ticker)



In [26]:
# Obter informações de velas para BTC/USDT
candlesticks = client.get_klines(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_1HOUR)
# Converter os dados em DataFrame
columns = [
    "Open Time", "Open", "High", "Low", "Close", "Volume",
    "Close Time", "Quote Asset Volume", "Number of Trades",
    "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"
]

df = pd.DataFrame(candlesticks, columns=columns)


In [27]:
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
df.drop(columns="Ignore", inplace=True)
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume
0,2024-10-20 06:00:00,68213.99000000,68410.00000000,68176.26000000,68400.01000000,446.43758000,2024-10-20 06:59:59.999,30478570.48719230,33944,348.67231000,23806563.19785060
1,2024-10-20 07:00:00,68400.01000000,68433.50000000,68364.10000000,68378.00000000,580.54038000,2024-10-20 07:59:59.999,39707874.10378670,36540,395.74384000,27068190.75434840
2,2024-10-20 08:00:00,68378.01000000,68500.00000000,68355.01000000,68394.00000000,454.72661000,2024-10-20 08:59:59.999,31109027.30928110,35691,231.42970000,15834468.67299280
3,2024-10-20 09:00:00,68393.99000000,68445.99000000,68355.00000000,68434.20000000,239.27437000,2024-10-20 09:59:59.999,16367001.59788360,27793,123.24928000,8430202.36842220
4,2024-10-20 10:00:00,68434.20000000,68458.30000000,68376.04000000,68376.05000000,241.82127000,2024-10-20 10:59:59.999,16546542.94039370,25376,119.08406000,8148129.80274320
...,...,...,...,...,...,...,...,...,...,...,...
495,2024-11-09 21:00:00,76293.99000000,76486.00000000,76227.52000000,76451.60000000,408.62709000,2024-11-09 21:59:59.999,31203906.69097480,74679,200.90652000,15343044.54172780
496,2024-11-09 22:00:00,76451.61000000,76606.34000000,76365.25000000,76519.90000000,610.63157000,2024-11-09 22:59:59.999,46703949.17892150,116587,324.31974000,24806897.36901940
497,2024-11-09 23:00:00,76519.90000000,76900.00000000,76474.00000000,76677.46000000,1773.62628000,2024-11-09 23:59:59.999,136107955.06038610,233421,1062.47265000,81540348.87672860
498,2024-11-10 00:00:00,76677.46000000,77100.00000000,76492.00000000,76884.01000000,1848.55068000,2024-11-10 00:59:59.999,141991790.93552590,218040,1197.83265000,92030909.53812990


In [28]:
for column in df.select_dtypes(include=['object']).columns:
    try:
        df[column] = df[column].astype(float)
    except ValueError as e:
        print(f"Erro ao converter a coluna {column}: {e}")


In [ ]:
df.describe()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume
count,500,500.000000,500.000000,500.00000,500.000000,500.000000,500,5.000000e+02,500.000000,500.000000,5.000000e+02
mean,2024-10-30 15:30:00,70112.799320,70341.861240,69898.74744,70129.739180,1295.109580,2024-10-30 16:29:59.999000064,9.187938e+07,172303.648000,642.930523,4.563803e+07
min,2024-10-20 06:00:00,65723.310000,65989.970000,65260.00000,65723.320000,110.328570,2024-10-20 06:59:59.999000,7.409048e+06,20741.000000,49.654790,3.334444e+06
25%,2024-10-25 10:45:00,67718.302500,67860.000000,67487.97000,67718.305000,561.273450,2024-10-25 11:44:59.999000064,3.884971e+07,83160.250000,271.886073,1.873801e+07
50%,2024-10-30 15:30:00,68970.030000,69181.275000,68740.63000,68974.135000,871.070495,2024-10-30 16:29:59.999000064,6.066450e+07,131548.000000,419.221975,2.964128e+07
75%,2024-11-04 20:15:00,72274.507500,72441.407500,72064.73500,72277.537500,1505.080620,2024-11-04 21:14:59.999000064,1.068638e+08,220619.500000,747.618623,5.213905e+07
max,2024-11-10 01:00:00,76912.580000,77199.990000,76699.05000,76912.580000,13585.407137,2024-11-10 01:59:59.999000,1.007137e+09,753221.000000,7607.417947,5.639445e+08
std,NaN,3122.557001,3155.039397,3108.00969,3135.190003,1293.520498,NaN,9.433466e+07,124184.224271,679.246214,4.963940e+07
